In [1]:
%pip install stable-baselines3==2.0.0 tensorboard

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing necessary libraries
import gymnasium as gym
import mobile_env
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_checker import check_env

# predefined small scenarios
from mobile_env.scenarios.smart_city import MComSmartCity

# easy access to the default configuration
MComSmartCity.default_config()

env = gym.make("mobile-smart_city-smart_city_handler-v0", render_mode="rgb_array")

/Users/elifohri/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# predefined smart city scenario
from mobile_env.scenarios.smart_city import MComSmartCity

# easy access to the default smart city configuration, parameters and settings
MComSmartCity.default_config()

{'width': 200,
 'height': 200,
 'EP_MAX_TIME': 100,
 'seed': 666,
 'reset_rng_episode': False,
 'arrival': mobile_env.core.arrival.NoDeparture,
 'channel': mobile_env.core.channels.OkumuraHata,
 'scheduler': mobile_env.core.schedules.ResourceFair,
 'movement': mobile_env.core.movement.RandomWaypointMovement,
 'utility': mobile_env.core.utilities.BoundedLogUtility,
 'sensor_placement': mobile_env.core.sensor_placement.SensorPlacement,
 'handler': mobile_env.handlers.smart_city_handler.MComSmartCityHandler,
 'bs': {'bw': 100000000.0,
  'freq': 3500,
  'tx': 40,
  'height': 40,
  'computational_power': 100},
 'ue': {'velocity': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09, 'height': 1.5},
 'sensor': {'height': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09},
 'ue_job': {'job_generation_probability': 0.7,
  'communication_job_lambda_value': 50.0,
  'computation_job_lambda_value': 5.0},
 'sensor_job': {'communication_job_lambda_value': 70.0,
  'computation_job_lambda_value': 7.0},
 'e2e_delay_threshold': 2.0

In [4]:
import random

class CustomEnv(MComSmartCity):
    # overwrite the default config
    @classmethod
    def default_config(cls):
        config = super().default_config()
        config.update({"seed": random.randint(0, 10000),})
        return config

    def __init__(self, config={}, render_mode=None):
        # load default config defined above; overwrite with custom params
        env_config = self.default_config()
        env_config.update(config)

        super().__init__(config, render_mode)

In [5]:
# Create environment once
env = CustomEnv(render_mode="rgb_array")
env.reset()

(array([0., 0.], dtype=float32),
 {'time': 0.0,
  'total reward': 0.0,
  'total delayed UE jobs': 0.0,
  'total throughput ue': <function mobile_env.core.metrics.calculate_cumulative_throughput_ue(sim)>,
  'total throughput sensor': <function mobile_env.core.metrics.calculate_cumulative_throughput_sensor(sim)>,
  'total throughput e2e ue': <function mobile_env.core.metrics.calculate_cumulative_throughput_e2e_ue(sim)>,
  'total throughput e2e sensor': <function mobile_env.core.metrics.calculate_cumulative_throughput_e2e_sensor(sim)>,
  'total e2e delay': 0.0,
  'total synchronization delay': 0.0,
  'bs trans. ue': {0: 0},
  'bs trans. ss': {0: 0},
  'bs accomp. ue': {0: 0},
  'bs accomp. ss': {0: 0}})

In [6]:
# Default configuration
env.default_config()

{'width': 200,
 'height': 200,
 'EP_MAX_TIME': 100,
 'seed': 2566,
 'reset_rng_episode': False,
 'arrival': mobile_env.core.arrival.NoDeparture,
 'channel': mobile_env.core.channels.OkumuraHata,
 'scheduler': mobile_env.core.schedules.ResourceFair,
 'movement': mobile_env.core.movement.RandomWaypointMovement,
 'utility': mobile_env.core.utilities.BoundedLogUtility,
 'sensor_placement': mobile_env.core.sensor_placement.SensorPlacement,
 'handler': mobile_env.handlers.smart_city_handler.MComSmartCityHandler,
 'bs': {'bw': 100000000.0,
  'freq': 3500,
  'tx': 40,
  'height': 40,
  'computational_power': 100},
 'ue': {'velocity': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09, 'height': 1.5},
 'sensor': {'height': 1.5, 'snr_tr': 2e-08, 'noise': 1e-09},
 'ue_job': {'job_generation_probability': 0.7,
  'communication_job_lambda_value': 50.0,
  'computation_job_lambda_value': 5.0},
 'sensor_job': {'communication_job_lambda_value': 70.0,
  'computation_job_lambda_value': 7.0},
 'e2e_delay_threshold': 2.

In [7]:
# Load the trained model
model_path1 = "/Users/elifohri/Documents/mobile-env/example_scenarios/trainings_with_different_seed_new_throughput/df_0.9/ppo_models/ppo_seed_666.zip"
model1 = PPO.load(model_path1)

/Users/elifohri/Library/Python/3.9/lib/python/site-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file

In [ ]:
metrics_df

In [ ]:
all_synch_delays

In [ ]:
import matplotlib.pyplot as plt

# Plot the distribution of actions
plt.figure(figsize=(10, 6))
plt.hist(all_actions[:, 0], bins=20, alpha=0.7, label='Action 1')
plt.hist(all_actions[:, 1], bins=20, alpha=0.7, label='Action 2')
plt.title('Action Distribution')
plt.xlabel('Action Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Calculate basic statistics for observations
obs_mean = np.mean(all_observations, axis=0)
obs_std = np.std(all_observations, axis=0)
obs_min = np.min(all_observations, axis=0)
obs_max = np.max(all_observations, axis=0)

print(f"Mean of Observations: {obs_mean}")
print(f"Standard Deviation of Observations: {obs_std}")
print(f"Minimum of Observations: {obs_min}")
print(f"Maximum of Observations: {obs_max}")

# Plot the distribution of observations
plt.figure(figsize=(10, 6))
plt.hist(all_observations[:, 0], bins=20, alpha=0.7, label='Observation 1')
plt.hist(all_observations[:, 1], bins=20, alpha=0.7, label='Observation 2')
plt.title('Observation Distribution')
plt.xlabel('Observation Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()

In [ ]:
# Calculate basic statistics for rewards
reward_mean = np.mean(all_rewards)
reward_std = np.std(all_rewards)
reward_min = np.min(all_rewards)
reward_max = np.max(all_rewards)

print(f"Mean of Rewards: {reward_mean}")
print(f"Standard Deviation of Rewards: {reward_std}")
print(f"Minimum of Rewards: {reward_min}")
print(f"Maximum of Rewards: {reward_max}")

# Plot the distribution of rewards
plt.figure(figsize=(10, 6))
plt.hist(all_rewards, bins=20, alpha=0.7, color='blue', label='Rewards')
plt.title('Reward Distribution')
plt.xlabel('Reward Value')
plt.ylabel('Frequency')
plt.legend()
plt.show()